In [1]:
# 混合前端的seq2seq模型部署

# 本教程将介绍如何是seq2seq模型转换为PyTorch可用的前端混合Torch脚本。我们要转换的模型来自于聊天机器人教程Chatbot tutorial。

# 1.混合前端
# 在一个基于深度学习项目的研发阶段, 使用像PyTorch这样即时eager、命令式的界面进行交互能带来很大便利。 
# 这使用户能够在使用Python数据结构、控制流操作、打印语句和调试实用程序时通过熟悉的、惯用的Python脚本编写。

# 尽管即时性界面对于研究和试验应用程序是一个有用的工具，但是对于生产环境中部署模型时，使用基于图形graph-based的模型表示将更加适用的。 
# 一个延迟的图型展示意味着可以优化，比如无序执行操作，以及针对高度优化的硬件架构的能力。 
# 此外，基于图形的表示支持框架无关的模型导出。PyTorch提供了将即时模式的代码增量转换为Torch脚本的机制，
# Torch脚本是一个在Python中的静态可分析和可优化的子集，Torch使用它来在Python运行时独立进行深度学习。

# 在Torch中的torch.jit模块可以找到将即时模式的PyTorch程序转换为Torch脚本的API。 
# 这个模块有两个核心模式用于将即时模式模型转换为Torch脚本图形表示: 跟踪tracing以及脚本化scripting。
# torch.jit.trace 函数接受一个模块或者一个函数和一组示例的输入，然后通过函数或模块运行输入示例，
# 同时跟跟踪遇到的计算步骤，然后输出一个可以展示跟踪流程的基于图形的函数。
# 跟踪Tracing对于不涉及依赖于数据的控制流的直接的模块和函数非常有用，就比如标准的卷积神经网络。

# 然而，如果一个有数据依赖的if语句和循环的函数被跟踪，则只记录示例输入沿执行路径调用的操作。
# 换句话说，控制流本身并没有被捕获。要将带有数据依赖控制流的模块和函数进行转化，已提供了一个脚本化机制。
# 脚本显式地将模块或函数代码转换为Torch脚本，包括所有可能的控制流路径。 如需使用脚本模式script mode， 
# 要确定继承了 torch.jit.ScriptModule基本类 (取代torch.nn.Module) 并且增加 torch.jit.script 装饰器到你的Python函数或者 
# torch.jit.script_method 装饰器到你的模块方法。

# 使用脚本化的一个警告是，它只支持Python的一个受限子集。要获取与支持的特性相关的所有详细信息，
# 请参考 Torch Script language reference。为了达到最大的灵活性，可以组合Torch脚本的模式来表示整个程序，
# 并且可以增量地应用这些技术。 
# 混合前端的流程图：图片缺


In [15]:
# 2 预备环境

# 首先，导入所需的模块以及设置一些常量。如果想使用自己的模型，需要保证MAX_LENGTH常量设置正确。 
# 提醒：这个常量定义了在训练过程中允许的最大句子长度以及模型能够产生的最大句子长度输出。
# source-python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.jit.ScriptModule
import re
import os
import unicodedata
import numpy as np

device = torch.device("cpu")

MAX_LENGTH = 10 # Maximum sentence length

# 默认的词向量
PAD_token = 0 # Used for padding short sentences
SOS_token = 1 # Start-of-sentence token
EOS_token = 2 # End-of-sentence token

ModuleNotFoundError: No module named 'torch.jit.ScriptModule'

In [3]:
# 3.模型概述

# 正如前文所言，我们使用的sequence-to-sequence (seq2seq) 模型。
# 这种类型的模型用于输入是可变长度序列的情况，我们的输出也是一个可变长度序列，它不一定是一对一输入映射。
# seq2seq 模型由两个递归神经网络(RNNs)组成：编码器 encoder和解码器decoder. 
# 模型概述图片：图片缺

# （1）编码器(Encoder)
# 编码器RNN在输入语句中每次迭代一个标记(例如单词)，每次步骤输出一个“输出”向量和一个“隐藏状态”向量。
# ”隐藏状态“向量在之后则传递到下一个步骤，同时记录输出向量。
# 编码器将序列中每个坐标代表的文本转换为高维空间中的一组坐标，解码器将使用这些坐标为给定的任务生成有意义的输出。

# （2）解码器(Decoder)
# 解码器RNN以逐个令牌的方式生成响应语句。它使用来自于编码器的文本向量和内部隐藏状态来生成序列中的下一个单词。
# 它继续生成单词，直到输出表示句子结束的EOS语句。我们在解码器中使用专注机制attention mechanism来帮助它
# 在输入的某些部分生成输出时"保持专注"。对于我们的模型，我们实现了Luong et al等人的“全局关注Global attention”模块，
# 并将其作为解码模型中的子模块。

In [4]:
# 4.数据处理

# 尽管我们的模型在概念上处理标记序列，但在现实中，它们与所有机器学习模型一样处理数字。
# 在这种情况下，在训练之前建立的模型词汇表中的每个单词都映射到一个整数索引。
# 我们使用Voc对象来包含从单词到索引的映射，以及词汇表中的单词总数。我们将在运行模型之前加载对象。

# 此外，为了能够进行评估，我们必须提供一个处理字符串输入的工具。normalizeString函数将字符串中的所有字符转换为小写，
# 并删除所有非字母字符。indexesFromSentence函数接受一个单词的句子并返回相应的单词索引序列。

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # 统计SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words # 对每个word依次分配index
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k) #这样处理后，词频信息丢掉了

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))
        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # 统计默认的令牌
        for word in keep_words:
            self.addWord(word) # trimmed后，word对应的index可能发生变化，但其实无所谓....

# 小写并删除非字母字符
def normalizeString(s):
    s = s.lower()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

# 使用字符串句子，返回单词索引的句子
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [5]:
# 5.定义编码器

# 这里大概说一下pytorch的RNN的一些用法，参考： https://www.jianshu.com/p/b942e65cb0a3
# Torch中的CNN和RNN中的batchSize的默认位置是不同的。
# CNN中：batchsize的位置是position 0.
# RNN中：batchsize的位置是position 1.

# 在RNN中输入数据格式：
# 对于最简单的RNN，我们可以使用两种方式来调用,torch.nn.RNNCell(),它只接受序列中的单步输入，必须显式的传入隐藏状态。
# torch.nn.RNN()可以接受一个序列的输入，默认会传入一个全0的隐藏状态，也可以自己申明隐藏状态传入。

# 输入大小是三维tensor[seq_len,batch_size,input_dim]
# input_dim是输入的维度，比如是128   通常tokenizer之后embeding的维度
# batch_size是一次往RNN输入句子的数目，比如是5。
# seq_len是一个句子的最大长度，比如15   通常是定义一个最大的长度，不足的部分补一个特殊的token，比如TOKEN_PAD
# 所以千万注意，RNN输入的是序列，一次把批次的所有句子都输入了，得到的ouptut和hidden都是这个批次的所有的输出和隐藏状态，维度也是三维。
# **可以理解为现在一共有batch_size个独立的RNN组件，RNN的输入维度是input_dim，
# 总共输入seq_len个时间步，则每个时间步输入到这个整个RNN模块的维度是[batch_size,input_dim]
# 例如：
# 构造RNN网络，x的维度5，隐层的维度10,网络的层数2
# rnn_seq = nn.RNN(5, 10, 2)  
# 构造一个输入序列，句长为 6，batch 是 3， 每个单词使用长度是 5的向量表示
# x = torch.randn(6, 3, 5) # 6是seq_len，3是batch_size, 5是input_dim和上面的5要对应起来
# out,ht = rnn_seq(x) #h0可以指定或者不指定，或者 out,ht = rnn_seq(x,h0) 指定h0

# 问题1：这里out、ht的size是多少呢？
# 回答：out:6 * 3 * 10, ht: 2 * 3 * 10，
# out的输出维度[seq_len, batch_size, output_dim]，ht的维度[num_layers * num_directions, batch, hidden_size],
# 如果是单向单层的RNN那么一个句子只有一个hidden。num_directions应该是单向的是1，双向的是2
# liujia: 这里的output_dim应该和hidden_size一样吧。。。。见问题2，就是一样的
# liujia: out的输出维度，最后一维应该是out_dim * n_direction?

# 问题2：out[-1]和ht[-1]是否相等？
# 回答：相等，隐藏单元就是输出的最后一个单元，可以想象，每个的输出其实就是那个时间步的隐藏单元

# RNN的其它参数
# RNN(input_dim ,hidden_dim ,num_layers ，…)
# – input_dim 表示输入的特征维度
# – hidden_dim 表示输出的特征维度，如果没有特殊变化，相当于out
# – num_layers 表示网络的层数  liujia:多层相当于stack那种架构
# – nonlinearity 表示选用的非线性激活函数，默认是 ‘tanh’
# – bias 表示是否使用偏置，默认使用  liujia: 这个是什么？输出的out是hidden state经过变换而来的，这个是变换的偏置么？
# – batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位
# – dropout 表示是否在输出层应用 dropout   liujia: dropout怎么做？
# – bidirectional 表示是否使用双向的 rnn，默认是 False

# LSTM的输出多了一个memory单元
# 例如：
# 输入维度 50，隐层100维，两层
# lstm_seq = nn.LSTM(50, 100, num_layers=2)
# 输入序列seq=10，batch=3，输入维度=50
# lstm_input = torch.randn(10, 3, 50)
# out, (h, c) = lstm_seq(lstm_input) # 使用默认的全0隐藏状态

# 问题1：out和(h,c)的size各是多少？
# 回答：out：(10 * 3 * 100)，(h,c)：都是(2 * 3 * 100) 和上面的传统RNN一样。。。
# 问题2：out[-1,:,:]和h[-1,:,:]相等吗？
# 回答： 相等

# GRU比较像传统的RNN
# gru_seq = nn.GRU(10, 20,2) # x_dim,h_dim,layer_num
# gru_input = torch.randn(3, 32, 10) # seq，batch，x_dim
# out, h = gru_seq(gru_input)

# ---------------------------------------------------------------------------------------
# 通过torch.nn.GRU模块实现编码器的RNN。本模块接受一批语句(嵌入单词的向量)的输入，它在内部遍历这些句子，
# 每次一个标记，计算隐藏状态。我们将这个模块初始化为双向的，
# 这意味着我们有两个独立的GRUs:一个按时间顺序遍历序列，另一个按相反顺序遍历序列。 
# 我们最终返回这两个GRUs输出的和。由于我们的模型是使用批处理进行训练的，
# 所以我们的EncoderRNN模型的forward函数需要一个填充的输入批处理。为了批量处理可变长度的句子，
# 我们通过MAX_LENGTH令牌允许一个句子中支持的最大长度，并且批处理中所有小于MAX_LENGTH
# 令牌的句子都使用我们专用的PAD_token令牌填充在最后。要使用带有PyTorch RNN模块的批量填充，
# 我们必须把转发forward密令在 调用torch.nn.utils.rnn.pack_padded_sequence
# 和torch.nn.utils.rnn.pad_packed_sequence数据转换时进行打包。注意，forward 函数还接受一个input_length列表，
# 其中包含批处理中每个句子的长度。该输入在填充时通过torch.nn.utils.rnn.pack_padded_sequence使用。

# 混合前端笔记: 由于编码器的转发函数forward不包含任何依赖于数据的控制流，
# 因此我们将使用跟踪tracing将其转换为脚本模式script mode。
# 在跟踪模块时，我们可以保持模块定义不变。在运行评估之前，我们将在本文末尾初始化所有模型。

class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # 初始化GRU;input_size和hidden_size参数都设置为'hidden_size'
        # 因为我们输入的大小是一个有多个特征的词向量== hidden_size
        self.gru = nn.GRU(hidden_size,   #input_dim设置为何hidden_size一样
                          hidden_size, 
                          n_layers,
                          dropout=(0 if n_layers == 1 else dropout), 
                          bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # 将单词索引转换为向量
        embedded = self.embedding(input_seq)
        # 为RNN模块填充批次序列   
        # liujia: pack_padded_sequence为填充，pad_packed_sequence为对输出再去掉填充
        # 不过input_lengths说需要按照从大到小排列，而且第一个参数embedded需要也是这种顺序排列好的，是这样么？
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # 正向通过GRU
        outputs, hidden = self.gru(packed, hidden) #hidden可传可不传，不传就是默认的0
        # 打开填充
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # 将双向GRU的输出结果总和
        # liujia: 如果多层+双向，那么outputs的size是？？？
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # 返回输出以及最终的隐藏状态
        return outputs, hidden

In [6]:
# 6.定义解码器的注意力模块

# 接下来，将定义注意力模块(Attn)。请注意，此模块将用作解码器模型中的子模块。
# Luong等人考虑了各种“分数函数”score functions，它们取当前解码器RNN输出和整个编码器输出，
# 并返回关注点“能值”engergies。这个关注能值张量attension energies tensor与编码器输出的大小相同，
# 两者最终相乘，得到一个加权张量，其最大值表示在特定时间步长解码的查询语句最重要的部分。

# Luong的注意力层
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # 根据给定的方法计算注意力权重（能量）
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # 转置max_length和batch_size维度
        attn_energies = attn_energies.t()

        # 返回softmax归一化概率分数（增加维度）
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [7]:
# 7.定义解码器

# 类似于EncoderRNN，我们使用torch.nn.GRU模块作为我们的解码器RNN。
# 然而，这一次我们使用单向GRU。需要注意的是，与编码器不同，我们将向解码器RNN每次提供一个单词。
# 我们首先得到当前单词的嵌入并应用抛出功能dropout。接下来，我们将嵌入和最后的隐藏状态转发给GRU，
# 得到当前的GRU输出和隐藏状态。然后，我们使用Attn模块作为一个层来获得专注权重，
# 我们将其乘以编码器的输出来获得我们的参与编码器输出。我们使用这个参与编码器输出作为文本context张量，
# 它表示一个加权和，表示编码器输出的哪些部分需要注意。 在这里，我们使用线性层linear layer
# 和softmax normalization归一化来选择输出序列中的下一个单词。

# *混合前端笔记 与EncoderRNN类似，此模块不包含任何依赖于数据的控制流。
# 因此，在初始化该模型并加载其参数之后，我们可以再次使用跟踪tracing将其转换为Torch脚本。

class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 保持参考
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 定义层
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 注意：我们这步只运行一次
        # 获取当前输入字对应的向量映射
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # 通过单向GRU转发
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 通过当前GRU的输出计算注意力权重
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # 注意力权重乘以编码器输出以获得新的“加权和”上下文向量
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # 使用Luong的公式5来连接加权上下文向量和GRU输出
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # 使用Luong的公式6来预测下一个单词
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # 返回输出和最终的隐藏状态
        return output, hidden

In [9]:
# 8.定义评估

# 8.1 贪婪搜索解码器
# 在聊天机器人教程中，我们使用GreedySearchDecoder模块来简化实际的解码过程。
# 该模块将训练好的编码器和解码器模型作为属性，驱动输入语句(词索引向量)的编码过程，并一次一个词(词索引)迭代地解码输出响应序列。

# 对输入序列进行编码很简单:只需将整个序列张量及其对应的长度向量转发给编码器。需要注意的是，
# 这个模块一次只处理一个输入序列，而不是成批的序列。因此，当常数1用于声明张量大小时，
# 它对应于批处理大小为1。要解码给定的解码器输出，我们必须通过解码器模型 迭代地向前运行，
#该解码器模型输出softmax分数，该分数对应于每个单词在解码序列中是正确的下一个单词的概率。
# 我们将decoder_input初始化为一个包含SOS_token的张量。在每次通过解码器之后，
# 我们贪婪地将softmax概率最高的单词追加到decoded_words列表中。 
# 我们还使用这个单词作为下一个迭代的decoder_input。如果decoded_words列表的长度达到MAX_LENGTH，
# 或者预测的单词是EOS_token，那么解码过程将终止。

# ```buildoutcfg
class GreedySearchDecoder(torch.jit.ScriptModule): 
    def init(self, encoder, decoder, decoder_n_layers): 
        super(GreedySearchDecoder, self).init() 
        self.encoder = encoder 
        self.decoder = decoder 
        self._device = device 
        self._SOS_token = SOS_token 
        self._decoder_n_layers = decoder_n_layers

    constants = ['_device', '_SOS_token', '_decoder_n_layers']

    @torch.jit.script_method 
    def forward(self, input_seq : torch.Tensor, input_length : torch.Tensor, max_length : int): 
        # 通过编码器模型转发输入 
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length) 
        # 准备编码器的最终隐藏层作为解码器的第一个隐藏输入 
        decoder_hidden = encoder_hidden[:self._decoder_n_layers] 
        # 使用SOS_token初始化解码器输入 
        decoder_input = torch.ones(1, 1, device=self._device, dtype=torch.long) * self._SOS_token 
        # 初始化张量以将解码后的单词附加到 
        all_tokens = torch.zeros([0], device=self._device, dtype=torch.long) 
        all_scores = torch.zeros([0], device=self._device) 
        # 一次迭代地解码一个词令牌 
        for _ in range(max_length): 
            # 正向通过解码器 
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs) 
            # 获得最可能的单词标记及其softmax分数 
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1) 
            # 记录令牌和分数 
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0) 
            all_scores = torch.cat((all_scores, decoder_scores), dim=0) 
            # 准备当前令牌作为下一个解码器输入（添加维度） 
            decoder_input = torch.unsqueeze(decoder_input, 0) 
            
        # 返回词令牌和分数的集合 
        return all_tokens, all_scores

ValueError: substring not found

In [10]:
# 8.2 输入评估

# 接下来，我们定义一些函数来计算输入。求值函数`evaluate`接受一个规范化字符串语句，将其处理为其对应的单词索引张量(批处理大小
# 为1)，并将该张量传递给一个名为`searcher`的`GreedySearchDecoder`实例，以处理编码/解码过程。检索器返回输出的单词索引向量和
# 一个分数张量，该张量对应于每个解码的单词标记的`softmax`分数。最后一步是使用`voc.index2word`将每个单词索引转换回其字符串表示形式。

# 我们还定义了两个函数来计算输入语句。`evaluateInput`函数提示用户输入，并计算输入。它持续请求另一次输入，直到用户输入“q”或“quit”。

# `evaluateExample`函数只接受一个字符串输入语句作为参数，对其进行规范化、计算并输出响应。
# ```buildoutcfg
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    # 格式化输入句子作为批处理
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # 创建长度张量
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # 转置批量的维度以匹配模型的期望
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 使用适当的设备
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # 用earcher解码句子s
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


# 评估来自用户输入的输入(stdin)
def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 获取输入的句子
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 规范化句子
            input_sentence = normalizeString(input_sentence)
            # 评估句子
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 格式化和打印回复句
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

# 规范化输入句子并调用evaluate()
def evaluateExample(sentence, encoder, decoder, searcher, voc):
    print("> " + sentence)
    # 规范化句子
    input_sentence = normalizeString(sentence)
    # 评估句子
    output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    print('Bot:', ' '.join(output_words))

In [13]:
# 9.加载预训练参数

# 9.1 使用托管模型
# 托管模型使用步骤:
# 1.下载模型这里: https://download.pytorch.org/models/tutorials/4000_checkpoint.tar
# 2.设置loadFilename变量作为下载的检查点文件的路径 
# 3.将checkpoint = torch.load(loadFilename)行取消注释，表示托管模型在CPU上训练。

# 9.2 使用自己的模型
# 加载自己的预训练模型设计步骤:
# 1.将loadFilename变量设置为希望加载的检查点文件的路径。注意，如果您遵循从chatbot tutorial中保存模型的协议，
# 这会涉及更改 model_name、encoder_n_layers、decoder_n_layers、hidden_size和checkpoint_iter(因为这些值在模型路径中使用到)。 
# 2.如果你在CPU上训练，确保你在checkpoint = torch.load(loadFilename)行打开了检查点。
# 如果你在GPU上训练，并且在CPU运行这篇 教程，解除checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))的注释。

# 请注意，我们像往常一样初始化并将参数加载到编码器和解码器模型中。
# 另外，在跟踪模型之前，我们必须调用.to(device)来设置模型的设备选项，
# 调用.eval()来设置抛出层dropout layer为test mode。
# TracedModule对象不继承to或eval方法。

save_dir = os.path.join("data", "save")
corpus_name = "cornell movie-dialogs corpus"

# 配置模型
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# 如果你加载的是自己的模型
# 设置要加载的检查点
checkpoint_iter = 4000
# loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                             '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                             '{}_checkpoint.tar'.format(checkpoint_iter))

# 如果你加载的是托管模型
loadFilename = 'data/4000_checkpoint.tar'

# 加载模型
# 强制CPU设备选项（与本教程中的张量匹配）
checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
encoder_sd = checkpoint['en']
decoder_sd = checkpoint['de']
encoder_optimizer_sd = checkpoint['en_opt']
decoder_optimizer_sd = checkpoint['de_opt']
embedding_sd = checkpoint['embedding']

# liujia: 相当于直接初始化voc对象的各个值了，即直接赋值voc对象的__dict__
# python类的静态函数、类函数、普通函数、全局变量以及一些内置的属性都是放在类__dict__里的
# 对象的__dict__中存储了一些self.xxx的一些东西
# python里除了int等一些内置对象外，都有__dict__
voc = Voc(corpus_name)
voc.__dict__ = checkpoint['voc_dict']

#print(len(voc.__dict__))
#print(voc.__dict__)


print('Building encoder and decoder ...')
# 初始化词向量  liujia:
embedding = nn.Embedding(voc.num_words, hidden_size)
embedding.load_state_dict(embedding_sd)
# 初始化编码器和解码器模型
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
# 加载训练模型参数
encoder.load_state_dict(encoder_sd)
decoder.load_state_dict(decoder_sd)
# 使用适当的设备
encoder = encoder.to(device)
decoder = decoder.to(device)
# 将dropout层设置为eval模式
encoder.eval()
decoder.eval()
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [14]:
# 10.将模型转换为 Torch 脚本

# 10.1 编码器
# 正如前文所述，要将编码器模型转换为Torch脚本，我们需要使用跟踪Tracing。
# 跟踪任何需要通过模型的forward方法运行一个示例输入，以及跟踪数据相遇时的图形计算。
# 编码器模型接收一个输入序列和一个长度相关的张量。因此，我们创建一个输入序列test_seq，
# 配置合适的大小(MAX_LENGTH,1)包含适当范围内的数值([0,voc.num_words])以及搭配的类型(int64)。
# 我们还创建了test_seq_length 标量，该标量实际包含与test_seq中单词数量对应的值。
# 下一步是使用torch.jit.trace函数来跟踪模型。
# 注意，我们传递的第一个参数是要跟踪的模块，第二个参数是模块forward方法的参数元组。

# 10.2 解码器
# 我们对解码器的跟踪过程与对编码器的跟踪过程相同。
# 请注意，我们对traced_encoder的一组随机输入调用forward，以获得解码器所需的输出。
# 这不是必需的，因为我们也可以简单地生成一个形状、类型和值范围正确的张量。
# 这种方法是可行的，因为在我们的例子中，我们对张量的值没有任何约束，因为我们没有任何操作可能导致超出范围的输入出错。

# 10.3 贪婪搜索解码器
# 回想一下，由于存在依赖于数据的控制流，我们为搜索器模块编写了脚本。
# 在脚本化的情况下，我们通过添加修饰符并确保实现符合脚本规则来预先完成转换工作。
# 我们初始化脚本搜索器的方式与初始化未脚本化变量的方式相同。

### 转换编码器模型
# 创建人工输入
test_seq = torch.LongTensor(MAX_LENGTH, 1).random_(0, voc.num_words).to(device)
test_seq_length = torch.LongTensor([test_seq.size()[0]]).to(device)
# 跟踪模型
traced_encoder = torch.jit.trace(encoder, (test_seq, test_seq_length))

### 转换解码器模型
# 创建并生成人工输入
test_encoder_outputs, test_encoder_hidden = traced_encoder(test_seq, test_seq_length)
test_decoder_hidden = test_encoder_hidden[:decoder.n_layers]
test_decoder_input = torch.LongTensor(1, 1).random_(0, voc.num_words)
# 跟踪模型
traced_decoder = torch.jit.trace(decoder, (test_decoder_input, test_decoder_hidden, test_encoder_outputs))

### 初始化searcher模块
scripted_searcher = GreedySearchDecoder(traced_encoder, traced_decoder, decoder.n_layers)

NameError: name 'GreedySearchDecoder' is not defined